In [1]:
from cvxopt import solvers as cvxopt_solvers
from cvxopt import matrix as cvxopt_matrix
from sklearn import svm
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import numpy as np
from sklearn import svm
import scipy

import cvxpy as cp
cvxopt_solvers.options['show_progress'] = False

"""
import matplotlib
from sklearn import svm
import plotly.express as px
from scipy.sparse import csr_matrix 
from numpy import linalg as LA
"""


'\nimport matplotlib\nfrom sklearn import svm\nimport plotly.express as px\nfrom scipy.sparse import csr_matrix \nfrom numpy import linalg as LA\n'

In [2]:
col_name = [i for i in range(201)]
train = pd.read_csv('train.csv', sep = ',', names = col_name)
data_test = pd.read_csv('test.csv', sep = ',', names = col_name)
data_train = train.iloc[0:4000]
validation = train.iloc[4000:]

In [3]:
# describing data frame
def describe_data(df):
    print("statistical information: ")
    display(df.describe(include = 'all'))
    duplicates = df.duplicated().sum()
    if duplicates > 0:
        print(f'The number of data duplicates is {duplicates}')
    else:
        print('There are no duplicates in the data')
    print("Data contains {0} samples/rows and {1} columns/features".format(df.shape[0], df.shape[1] - 1))
    # print("Number of dimensions:",df.shape[1])

describe_data(data_train)

statistical information: 


,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,200
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.00000,...,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,0.510500,0.020805,-0.027538,0.007003,-0.002810,0.004585,0.020467,0.004732,0.000155,-0.00085,...,-0.002163,-0.007002,0.015572,-0.005530,0.037802,-0.013870,0.019555,-0.025915,0.007270,0.010985
std,0.499952,1.046795,1.055809,1.035735,1.051085,1.059428,1.046060,1.058164,1.057871,1.06592,...,1.045139,1.056644,1.075913,1.045216,1.039436,1.051253,1.071891,1.037967,1.057728,1.047149
min,0.000000,-3.870000,-4.720000,-3.570000,-3.350000,-4.230000,-3.470000,-3.560000,-3.650000,-3.53000,...,-4.140000,-4.340000,-4.040000,-3.530000,-3.480000,-3.930000,-3.920000,-3.620000,-4.080000,-4.260000
25%,0.000000,-0.680000,-0.740000,-0.670000,-0.710000,-0.700000,-0.690000,-0.710000,-0.710000,-0.72000,...,-0.710000,-0.730000,-0.700000,-0.730000,-0.660000,-0.730000,-0.702500,-0.740000,-0.690000,-0.690000
50%,1.000000,0.010000,-0.030000,0.010000,0.010000,0.020000,0.010000,0.000000,-0.010000,0.00000,...,-0.020000,-0.030000,0.020000,0.010000,0.040000,0.020000,0.030000,-0.015000,0.010000,0.030000
75%,1.000000,0.730000,0.672500,0.720000,0.710000,0.740000,0.750000,0.730000,0.730000,0.70000,...,0.700000,0.710000,0.750000,0.720000,0.740000,0.710000,0.740000,0.680000,0.710000,0.720000
max,1.000000,4.280000,3.510000,3.340000,3.600000,3.670000,3.580000,3.500000,4.020000,3.90000,...,3.950000,3.770000,3.600000,3.760000,3.390000,3.330000,4.210000,3.450000,4.050000,3.930000


There are no duplicates in the data
Data contains 4000 samples/rows and 200 columns/features


In [4]:
# preprocessing
label_train = data_train.values[:,0] # y becomes our labels vector
label_train[label_train == 0] = -1 # replacing 0 classes with -1
data_train = data_train.values[:, 1:]

label_test = data_test.values[:,0] # y becomes our labels vector
label_test[label_test == 0] = -1
data_test = data_test.values[:, 1:]



In [5]:
label_validation = validation.values[:,0] 
label_validation[label_validation == 0] = -1
data_validation = validation.values[:, 1:]

In [6]:
print(label_train)
print(data_train)

[-1.  1.  1. ...  1.  1.  1.]
[[-0.36 -0.91 -0.99 ...  0.3   2.44 -1.26]
 [-1.4  -1.9   0.09 ... -0.2  -0.92 -0.46]
 [-0.43  1.45 -0.68 ...  0.12  0.01 -0.56]
 ...
 [-0.57  0.14 -0.62 ...  0.    0.38 -0.82]
 [ 0.4   0.16 -0.49 ...  0.89  0.21  1.09]
 [ 0.86 -0.23 -2.01 ...  0.21  0.68  2.49]]


### question 2
Please implement the training and testing algorithms of soft-margin Linear Support Vector Machine from its primal form using CVX

By setting C = 100, run your implementation, please report the solution of   and sum of all dimensions of  solution, e.g., np.sum(w). (For a quick check of the correctness of your code)

In [7]:
def svm_train_primal(x , y , c):
    # number of samples/vectors and features/dimensions respectively
    n, d = x.shape 
    # 200 by 1 column vector
    w = cp.Variable(d) 
    # offset/bias
    b = cp.Variable()
    # xi (error/slack) variable for each sample corrects the distance between correct plane and input by placing an offset
    xi = cp.Variable(n)
    
    objective = cp.Minimize(0.5 * cp.square(cp.norm(w)) + (c / n) * cp.sum(xi))
    # find individual constraints y_i, x_i and xi_i for y_i(w.T @ x_i + b) >= 1 - xi_i
    constraint_1 = [y[i] * (x[i] @ w + b) >= 1 - xi[i] for i in range(n)]
    # x_i >= 0
    constraint_2 = [xi[i] >= 0 for i in range(n)]
    # combining constraints into cvxpy form
    constraints = constraint_1 + constraint_2
    problem = cp.Problem(objective, constraints)
    res = problem.solve()
    return (w.value, b.value, xi.value)


svm_primal_model = svm_train_primal(data_train, label_train, 100)

In [8]:

def show_values(svm_model):
    w, b, xi = svm_model
    # print(w)
    print("sum of all dimenisions w: ", np.sum(w))    
    print("primal value of b: ",b)
    
def svm_predict_primal(x , y , svm_model):
    w, b, xi = svm_model
    correct_predictions = 0
    test_size = x.shape[0]
    for i in range(test_size):
        y_pred = np.sign(np.dot(x[i], w) + b)
        if y_pred == y[i]:
            correct_predictions += 1
    return correct_predictions / test_size

show_values(svm_primal_model)
accuracy = svm_predict_primal(data_test, label_test, svm_primal_model)
print("accuracy of primal form is", accuracy * 100, "%")


sum of all dimenisions w:  -0.1452156803361282
primal value of b:  1.779813717087077
accuracy of primal form is 96.8 %


### Question 3
Please implement the training algorithm of the soft-margin Linear Support Vector Machine from its dual form:

In [9]:
# 1.15s training time
def svm_train_dual (x , y , c):
    # samples, dimensions     
    n,d = x.shape 
    y = y.reshape(-1,1) * 1.
    X_dash = y * x
    # calculate gram matrix H
    H = np.dot(X_dash , X_dash.T) * 1.

    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((n, 1)))
    G = cvxopt_matrix(np.vstack((np.eye(n)*-1,np.eye(n))))
    h = cvxopt_matrix(np.hstack((np.zeros(n), np.ones(n) * c)))
    # label vector of size n * 1, cvxopt_matrix(y, (1, n))
    A = cvxopt_matrix(y.reshape(1, -1))
    # scalar with value zeros
    b = cvxopt_matrix(np.zeros(1))

    # run solver by matching api
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)
    alphas = np.array(sol['x'])
    return alphas, y
    
svm_dual_model = svm_train_dual(data_train, label_train , 100)

In [10]:
def display_results(svm_model):
    alphas, y = svm_model
    print(alphas.shape)
    print("sum of all dimenisions alpha: ", np.sum(alphas))    
    
display_results(svm_dual_model)


(4000, 1)
sum of all dimenisions alpha:  19795.781131147898


### Question 4
Write code to obtain the primal problem solution w*, b* from its dual solution  alpha*


In [11]:
# primal problem solution from dual solution

def get_primal_solution_from_dual(x, svm_dual_model):
    alphas , y = svm_dual_model
    w = ((y * alphas).T @ x).T
    b = y - np.dot(x, w)
    b = np.mean(b) # take mean bias
    return w, b

def show(w, b):
    print("W* ", w)
    print("b*", b)
    
def svm_predict_dual(x, y, w, b):
    correct_predictions = 0
    test_size = x.shape[0]
    for i in range(test_size):
        y_pred = np.sign(np.dot(x[i,:], w) + b)
        if y_pred == y[i]:
            correct_predictions += 1
    return correct_predictions / test_size

w, b = get_primal_solution_from_dual(data_train, svm_dual_model)
dual_accuracy = svm_predict_dual(data_test, label_test, w, b)
show(w, b)
print("dual accuracy is ", dual_accuracy * 100, "%")


W*  [[ 1.36058445e-02]
 [-1.44494765e-01]
 [ 1.05602137e-01]
 [-1.01375042e-01]
 [ 1.60645124e-01]
 [-9.44365974e-02]
 [ 3.13664183e-02]
 [-1.27152307e-01]
 [ 1.43289557e-01]
 [ 2.59378428e-02]
 [ 2.05438001e-01]
 [ 1.39052047e-01]
 [-3.03939044e-02]
 [ 2.37242455e-01]
 [ 4.57591535e-03]
 [-1.58526876e-01]
 [-9.39414348e-02]
 [-9.76228613e-02]
 [-1.62307260e-01]
 [-9.40157756e-03]
 [ 6.42996647e-02]
 [ 1.60244533e-01]
 [ 1.15185126e-01]
 [-1.72891231e-02]
 [ 2.80434572e-02]
 [-8.10259517e-03]
 [-3.83024322e-03]
 [-5.72639351e-02]
 [-1.86879671e-01]
 [ 2.78171399e-01]
 [-9.10794539e-02]
 [-3.97619445e-02]
 [ 5.12808666e-02]
 [-1.64046621e-02]
 [ 1.98490167e-01]
 [-9.07271378e-02]
 [ 2.77654401e-02]
 [-1.13844387e-01]
 [ 1.54520656e-01]
 [-7.35715359e-02]
 [ 1.15954669e-02]
 [ 3.95013205e-02]
 [-1.61612940e-01]
 [-5.33783557e-03]
 [ 1.98252827e-02]
 [-4.62452243e-02]
 [ 2.47505618e-01]
 [ 7.97978534e-02]
 [ 1.72144634e-01]
 [ 1.94782877e-01]
 [-1.27425423e-01]
 [-3.64906470e-02]
 [ 6.863

### Question 5
Write code to find the support vectors from the primal problem solutions

equation form:  y_i = w.T * x_i + b = 1

In [12]:
x = data_test
w, b, _ = svm_primal_model
# y_pos 

x = 1 - b * (w)**-1
print(x)
w.T @ x.T + b >=  1
# y_neg
w.T @ x.T + b == - 1


[ 6.88654457e+01  1.65891434e+01 -4.15202965e+01  3.50409001e+01
 -2.25673566e+01  2.38523812e+02 -6.26840489e+01  1.75771985e+02
 -3.10857546e+02 -1.90217666e+03 -1.74809481e+01 -2.66079046e+01
 -2.49737514e+01 -2.74994940e+01 -9.64277256e+01  7.53788168e+01
  3.18007389e+01  4.04557323e+01  3.32462018e+01  7.00853810e+01
  8.39856566e+02 -3.96040384e+01  2.07531698e+02  3.95134849e+01
  8.99582297e+01  9.65187673e+01  5.01765978e+02 -1.87868681e+02
  2.51244897e+01 -1.44178945e+01  7.60512985e+01  1.30025667e+02
 -2.82443067e+01  8.49427161e+01 -1.94462324e+01  4.73718674e+01
 -7.78519843e+01  3.50306813e+01 -3.12836517e+01  4.90379633e+01
 -6.45054816e+01 -7.72795491e+01  7.64678953e+01  4.36101551e+02
 -6.38940291e+01  7.30194457e+01 -1.12008012e+01 -2.35055555e+01
 -1.13910769e+02 -1.70891578e+01  5.73908372e+01 -4.54371377e+01
 -1.95225342e+01  3.79557690e+01 -3.20717582e+01  4.35525029e+01
  3.21572605e+01 -1.74845926e+01  4.92631895e+01 -1.44883250e+00
 -7.37557336e+01  3.74176

False

### Question 6
Write code to find the support vectors from the dual problem solutions. Please copy the code snippet for the implementation

In [13]:
# x = linear space
x = data_train
w, b = get_primal_solution_from_dual(data_train, svm_dual_model)

# y_pos 
w.T * x + b == 1
# y_neg
w.T * x + b == -1

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

### Question 7
Write code to choose C by using the validation set. Please copy the code snippet for the implementation. Report the test accuracy you get by using the optimal C found in the validation set. 

In [14]:
def test_model_fit(x, y, w, b):
    test_size = x.shape[0]
    correct_predictions = 0
    for i in range(test_size):
        y_pred = np.sign(np.dot(x[i,:], w.T) + b)
        if y_pred == y[i]:
            correct_predictions += 1

    return correct_predictions / test_size

In [15]:
c_values = [2e-10, 2e-8, 2e-6, 2e-4, 2e-2, 2e0, 2e2, 2e4, 2e6,2e7,2e8,2e10]
max_accuracy = 0
optimal_c = 0

# start by taking the first 8 values for now as computation time is too slow;
for c in c_values[:6]:
    clf = svm.SVC(C = c, kernel = 'linear')
    clf.fit(data_validation, label_validation)
    w = clf.coef_
    b = clf.intercept_
    svm_accuracy = test_model_fit(data_train, label_train, w, b)
    if svm_accuracy > max_accuracy:
        max_accuracy = svm_accuracy
        optimal_c = c

    print("accuracy of scikit learn svm model is", svm_accuracy * 100, "% with c value", c)
    
print("most optimal c is:", optimal_c)

"""
use validation set
data_validation
label_validation
train model
get accuracy
store most accurate c """

accuracy of scikit learn svm model is 48.949999999999996 % with c value 2e-10
accuracy of scikit learn svm model is 48.949999999999996 % with c value 2e-08
accuracy of scikit learn svm model is 48.949999999999996 % with c value 2e-06
accuracy of scikit learn svm model is 96.89999999999999 % with c value 0.0002
accuracy of scikit learn svm model is 96.675 % with c value 0.02
accuracy of scikit learn svm model is 96.3 % with c value 2.0
most optimal c is: 0.0002


'\nuse validation set\ndata_validation\nlabel_validation\ntrain model\nget accuracy\nstore most accurate c '

In [16]:
# using primal form model
for c in c_values[:7]:
    svm_primal_model = svm_train_primal(data_train, label_train, 100)
    svm_accuracy = test_model_fit(data_train, label_train, w, b)
    if svm_accuracy > max_accuracy:
        max_accuracy = svm_accuracy
        optimal_c = c

    print("accuracy of scikit learn svm model is", svm_accuracy * 100, "% with c value", c)
    
print("most optimal c is:", optimal_c)


accuracy of scikit learn svm model is 96.3 % with c value 2e-10
accuracy of scikit learn svm model is 96.3 % with c value 2e-08
accuracy of scikit learn svm model is 96.3 % with c value 2e-06
accuracy of scikit learn svm model is 96.3 % with c value 0.0002
accuracy of scikit learn svm model is 96.3 % with c value 0.02
accuracy of scikit learn svm model is 96.3 % with c value 2.0
accuracy of scikit learn svm model is 96.3 % with c value 200.0
most optimal c is: 0.0002


### Question 8

In [17]:
optimal_C = 0.0002
# optimal_C = 100

clf = svm.SVC(C = optimal_C, kernel = 'linear')
clf.fit(data_train, label_train) # .fit(x, y)

def show(clf):
    # print('w = ',clf.coef_)
    print('b = ',clf.intercept_)
    print('Indices of support vectors = ', clf.support_)
    # print(clf.support_.shape)
    print('Support vectors = ', clf.support_vectors_)
    print('Number of support vectors for each class = ', clf.n_support_)
    print('Coefficients of the support vector in the decision function = ', np.abs(clf.dual_coef_))
    print(clf.support_vectors_.shape)

show(clf)

b =  [0.63642135]
Indices of support vectors =  [   0   28   43 ... 3993 3996 3998]
Support vectors =  [[-0.36 -0.91 -0.99 ...  0.3   2.44 -1.26]
 [ 1.01 -1.13  1.49 ...  0.23 -0.3  -0.01]
 [ 1.02  2.08 -0.72 ...  1.84 -0.13  1.25]
 ...
 [ 1.77 -1.64  0.66 ...  0.7   0.16 -0.97]
 [-0.42 -2.15  1.14 ... -0.44 -0.09 -0.35]
 [ 0.4   0.16 -0.49 ...  0.89  0.21  1.09]]
Number of support vectors for each class =  [754 760]
Coefficients of the support vector in the decision function =  [[0.0002 0.0002 0.0002 ... 0.0002 0.0002 0.0002]]
(1514, 200)


In [18]:
w = clf.coef_
b = clf.intercept_
svm_accuracy = test_model_fit(data_train, label_train, w, b)
print("accuracy of scikit learn svm model is", svm_accuracy * 100, "%")

accuracy of scikit learn svm model is 97.05 %
